In [1]:
import pyodbc
import pandas as pd

import warnings
warnings.filterwarnings('ignore') 

# defining the server and the database
server = '(localdb)\mssqllocaldb' 
database = 'AdventureWorks2012'  

# Define the connection string
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server}; \
    SERVER='+ server +'; \
    DATABASE ='+ database +';\
    Trusted_Connection=yes;'
) 

### Question 1 - AdventureWorks DB - ProductListPriceHistory & Product table
### 1. Get product name, end date, and list price
### (join product and ProductListPriceHistory tables on ProductID)

In [2]:
# Thought process:
# SELECT Product.Name, PriceHistory.EndDate, and PriceHistory.ListPrice
# JOIN Product and ProductListPriceHistory
# ON ProductID
# [AdventureWorks2012].[Production].[Product] AS Product
# [AdventureWorks2012].[Production].[ProductListPriceHistory] AS PriceHistory



# Performing a join to combine ProductListPriceHistory & Product table
# Using TOP 15 to limit results

sql_query = pd.read_sql_query("""SELECT TOP 15 Product.Name, PriceHistory.EndDate, PriceHistory.ListPrice 
                                 FROM [AdventureWorks2012].[Production].[Product] AS Product
                                 JOIN [AdventureWorks2012].[Production].[ProductListPriceHistory] AS PriceHistory
                                 ON Product.ProductID = PriceHistory.ProductID
                                 ORDER BY Product.Name, PriceHistory.EndDate""",conn)
sql_query


,Name,EndDate,ListPrice
0,All-Purpose Bike Stand,NaT,159.0000
1,AWC Logo Cap,NaT,8.9900
2,AWC Logo Cap,2006-06-30,8.6442
3,AWC Logo Cap,2007-06-30,8.6442
4,Bike Wash - Dissolver,NaT,7.9500
5,Cable Lock,2007-06-30,25.0000
6,Chain,NaT,20.2400
7,"Classic Vest, L",NaT,63.5000
8,"Classic Vest, M",NaT,63.5000
9,"Classic Vest, S",NaT,63.5000


### Question 2 - AdventureWorks DB - Person & PersonPhone table
### 2. Get phone numbers for all Persons who’s first name is Kim (join Person & PersonPhone)

In [3]:
# Thought Process:
# SELECT FULLNAME using FullName (Person.LastName, Person.FirstName), Phone.PhoneNumber
# JOIN [AdventureWorks2012].[Person].[Person] AS Person & [AdventureWorks2012].[Person].[PersonPhone] AS Phone table
# ON BusinessEntityID
# WHERE Person.FirstName = 'Kim'


# Performing a join to combine ProductListPriceHistory & Product table

sql_query = pd.read_sql_query("""SELECT Person.LastName + ', ' + Person.FirstName AS [Full Name], Phone.PhoneNumber
                                 FROM [AdventureWorks2012].[Person].[Person] AS Person
                                 JOIN [AdventureWorks2012].[Person].[PersonPhone] AS Phone 
                                 ON Person.BusinessEntityID = Phone.BusinessEntityID
                                 WHERE Person.FirstName = 'Kim'""",conn)
sql_query


,Full Name,PhoneNumber
0,"Abercrombie, Kim",334-555-0137
1,"Abercrombie, Kim",919-555-0100
2,"Abercrombie, Kim",208-555-0114
3,"Akers, Kim",440-555-0166
4,"Akers, Kim",791-555-0100
5,"Akers, Kim",508-555-0100
6,"Ralls, Kim",193-555-0100
7,"Ralls, Kim",197-555-0100
8,"Ralls, Kim",133-555-0100
9,"Ralls, Kim",309-555-0129


### Question 3 - AdventureWorks DB - BusinessEntityAddress,AddressType, Address, StateProvince
### 3. Get the Address Type, City, and State Code for Business entity 332 (Join
### [BusinessEntityAddress], [AddressType], Address, StateProvince)

In [4]:
# Thought Process:
# SELECT  AddressType, City, and State Code
# JOIN [AdventureWorks2012].[Person].[BusinessEntityAddress] AS Entity
# [AdventureWorks2012].[Person].[AddressType] AS Type
# ON AddressTypeID
# [AdventureWorks2012].[Person].[Address] AS Address
# ON AddressID
# [AdventureWorks2012].[Person].[StateProvince] AS State
# ON StateProvinceID
# WHERE Business entity 332


# Performing a join to combine ProductListPriceHistory & Product table

sql_query = pd.read_sql_query("""SELECT Entity.BusinessEntityID, Type.Name, Address.City, State.StateProvinceCode AS State
                                 FROM [AdventureWorks2012].[Person].[AddressType] AS Type
                                 INNER JOIN [AdventureWorks2012].[Person].[BusinessEntityAddress] AS Entity
                                 ON Type.AddressTypeID = Entity.AddressTypeID
                                 INNER JOIN [AdventureWorks2012].[Person].[Address] AS Address
                                 ON Entity.AddressID = Address.AddressID
                                 INNER JOIN [AdventureWorks2012].[Person].[StateProvince] AS State
                                 ON State.StateProvinceID = Address.StateProvinceID
                                 WHERE Entity.BusinessEntityID = 332""",conn)
sql_query


,BusinessEntityID,Name,City,State
0,332,Shipping,Montreal,QC
1,332,Main Office,Montreal,QC


### Question 4 - AdventureWorks DB - WorkOrder & ScrapReason
### 4. Count the Work orders by scrape reason (ScrapReason.name). Disregard
### scrap reasons with less than 50 work orders (Join ScrapReason &
### WorkOrder on ScrapReasonID)

In [5]:
# Thought Process:
# SELECT ScrapeReason.Name, COUNT(WorkOrder.WorkOrderID)
# JOIN [AdventureWorks2012].[Production].[WorkOrder] AS WorkOrder
# [AdventureWorks2012].[Production].[ScrapReason] AS ScrapReason
# ON ScrapeReasonID
# GROUP BY ScrapReasonID
# HAVING COUNT(WorkOrder.WorkOrderID) >= 50



# Performing a join to combine ScrapReason & WorkOrder table

sql_query = pd.read_sql_query("""SELECT ScrapReason.Name AS [Scrap Reason], COUNT(WorkOrder.WorkOrderID) AS [Amount Discarded]
                                 FROM [AdventureWorks2012].[Production].[WorkOrder] AS WorkOrder
                                 INNER JOIN [AdventureWorks2012].[Production].[ScrapReason] AS ScrapReason
                                 ON WorkOrder.ScrapReasonID = ScrapReason.ScrapReasonID
                                 GROUP BY ScrapReason.Name
                                 HAVING COUNT(WorkOrder.WorkOrderID) >= 50
                                 ORDER BY COUNT(WorkOrder.WorkOrderID) DESC""",conn)
sql_query



,Scrap Reason,Amount Discarded
0,Thermoform temperature too low,63
1,Gouge in metal,54
2,Stress test failed,52
3,Trim length too long,52
4,Wheel misaligned,51


### Question 5 - AdventureWorks DB - AddressType & ContactType table
### 5. Write a query that generates 3 copies out of each [AddressType] row
### (Hint: Cross Join to any table that has 3 records)

In [6]:
# Performing a cross join to combine AddressType & ContactType table

sql_query = pd.read_sql_query("""SELECT AddressType.*
                                 FROM [AdventureWorks2012].[Person].[AddressType]  AS AddressType
                                 CROSS JOIN [AdventureWorks2012].[Person].[ContactType] AS ContactType
                                 WHERE ContactType.ContactTypeID <= 3
                                 ORDER BY 1""",conn)
sql_query

,AddressTypeID,Name,rowguid,ModifiedDate
0,1,Billing,B84F78B1-4EFE-4A0E-8CB7-70E9F112F886,2002-06-01
1,1,Billing,B84F78B1-4EFE-4A0E-8CB7-70E9F112F886,2002-06-01
2,1,Billing,B84F78B1-4EFE-4A0E-8CB7-70E9F112F886,2002-06-01
3,2,Home,41BC2FF6-F0FC-475F-8EB9-CEC0805AA0F2,2002-06-01
4,2,Home,41BC2FF6-F0FC-475F-8EB9-CEC0805AA0F2,2002-06-01
5,2,Home,41BC2FF6-F0FC-475F-8EB9-CEC0805AA0F2,2002-06-01
6,3,Main Office,8EEEC28C-07A2-4FB9-AD0A-42D4A0BBC575,2002-06-01
7,3,Main Office,8EEEC28C-07A2-4FB9-AD0A-42D4A0BBC575,2002-06-01
8,3,Main Office,8EEEC28C-07A2-4FB9-AD0A-42D4A0BBC575,2002-06-01
9,4,Primary,24CB3088-4345-47C4-86C5-17B535133D1E,2002-06-01


### Question 6 - AdventureWorks DB - SalesOrderHeader & Customer table
### 6. Return customer IDs, their order IDs, and the order DT including
### customers who placed no orders. Only include account #s which start
### with AW0002981

In [7]:
# Thought Process:
# SELECT Customer.CustomerID, Header.OrderDate, Header.SalesOrderID, Customer.AccountNumber
# JOIN FROM [AdventureWorks2012].[Sales].[SalesOrderHeader] AS Header
# [AdventureWorks2012].[Sales].[Customer] AS Customer
# ON CustomerID
# WHERE Customer.AccountNumber LIKE 'AW0002981%'
# GROUP BY Customer.CustomerID


# Performing a cross left join to combine Customer & SalesOrderHeader table
# Using TOP 15 to limit results

sql_query = pd.read_sql_query("""SELECT TOP 15 Customer.CustomerID, Header.OrderDate, Header.SalesOrderID, 
                                 Customer.AccountNumber
                                 FROM [AdventureWorks2012].[Sales].[SalesOrderHeader] AS Header
                                 LEFT JOIN [AdventureWorks2012].[Sales].[Customer] AS Customer
                                 ON Header.CustomerID = Customer.CustomerID
                                 WHERE Customer.AccountNumber LIKE 'AW0002981%'
                                 ORDER BY Customer.CustomerID""",conn)
sql_query

,CustomerID,OrderDate,SalesOrderID,AccountNumber
0,29810,2005-09-01,44126,AW00029810
1,29810,2005-12-01,44792,AW00029810
2,29810,2006-03-01,45570,AW00029810
3,29810,2006-06-01,46379,AW00029810
4,29810,2006-09-01,47443,AW00029810
5,29810,2006-12-01,48381,AW00029810
6,29810,2007-03-01,49539,AW00029810
7,29810,2007-06-01,50750,AW00029810
8,29810,2007-09-01,53584,AW00029810
9,29810,2007-12-01,59019,AW00029810
